# Classification Demo

**Mount Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
!ls '/content/drive/My Drive/vision_intro_tutorial'

 2019-06-04-16:52:30--checkpoint.hdf5   2019-06-04-18:08:28--checkpoint.hdf5
 2019-06-04-17:14:09--checkpoint.hdf5   2019-06-04-18:08:28--model.json
 2019-06-04-17:14:51--checkpoint.hdf5   2019-06-04-19:05:11--checkpoint.hdf5
 2019-06-04-17:17:36--checkpoint.hdf5   2019-06-04-19:05:11--model.json
 2019-06-04-17:22:45--model.json       'asphalt data'
 2019-06-04-17:42:55--checkpoint.hdf5  'Computer Vision.gslides'
 2019-06-04-17:42:55--model.json        crackdetection_demo.ipynb
 2019-06-04-17:44:26--checkpoint.hdf5   requirements
 2019-06-04-17:44:26--model.json


**Install requirements**

In [3]:
!pip install -r '/content/drive/My Drive/vision_intro_tutorial/requirements'

**Import requirements**

In [4]:
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras import optimizers
import matplotlib.pylab as plt
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os.path
import sys
from sklearn.model_selection import train_test_split
import warnings
import datetime

np.random.seed(100)

Using TensorFlow backend.


**Some useful functions**

In [0]:
def snd_data_split_3(X,Y,train_proportion,test_proportion,validation_proportion):
    size_1 = 1 - train_proportion
    size_2 = validation_proportion/(test_proportion+validation_proportion)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=size_1)
    x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=size_2)
    return x_train,x_test,x_valid,y_train,y_test,y_valid

In [0]:
def class_accuracy_percentage_calc(Y_actual,Y_prediction):
    df_confusion = pd.crosstab(Y_actual, Y_prediction, rownames=['Actual'], colnames=['Predicted'], margins=True)
#    print(df_confusion)
    count_of_classess = df_confusion.All.size
#    print("No. of classes: ",count_of_classess-1)
    class_accuracy_percentage = []
    for i in range(count_of_classess-1):
        class_accuracy_percentage.append(float(df_confusion[i][i])/float(df_confusion.All[i]))
    return class_accuracy_percentage        

**Import data**

In [0]:
class_0 = []
class_1 = []
for root, dirs, files in os.walk('/content/drive/My Drive/vision_intro_tutorial/asphalt data/'+'NonCracks/'):
    for file in files:
        if file.endswith(".jpg"):
            file_path = os.path.join(root, file)
#             print(file_path)
            class_0.append(file_path)
for root, dirs, files in os.walk('/content/drive/My Drive/vision_intro_tutorial/asphalt data/'+'Cracks/'):
    for file in files:
        if file.endswith(".jpg"):
            file_path = os.path.join(root, file)
#             print(file_path)
            class_1.append(file_path)

In [8]:
df0 = pd.DataFrame(class_0)
df0['class'] = 0
df1 = pd.DataFrame(class_1)
df1['class'] = 1
dfs = [df0,df1]
df = pd.concat(dfs)
df = df.sample(frac=1).reset_index(drop=True)
df.shape

(400, 2)

**Load data**

In [9]:
X_files = df[0][:]
Y = df['class'][:]

X = np.zeros(shape=(X_files.shape[0],100,100,3))
for i in range(0,X_files.shape[0]):
    if os.path.isfile(X_files[i]):
        img = cv2.imread(X_files[i])
        img = cv2.resize(img, (100,100))
        blur = cv2.bilateralFilter(img,9,75,75)
        X[i, :, :, :]=blur
Y = Y.as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [0]:
x_train,x_test,x_valid,y_train,y_test_r,y_valid_r = snd_data_split_3(X,Y,train_proportion=0.7,test_proportion=0.1,validation_proportion=0.2)

In [0]:
x_train /= 255
x_test /= 255
x_valid /= 255

In [12]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_valid.shape[0], 'validation samples')

x_train shape: (279, 100, 100, 3)
279 train samples
40 test samples
81 validation samples


Turn data to categorical (it is necessary if we are going to have more that 2 classes)

In [0]:
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test_r, 2)
y_valid = keras.utils.to_categorical(y_valid_r, 2)

Augmentation

In [14]:
Augment = False
if Augment is False:
    print("skipping augmentation")
else:
    datagen = ImageDataGenerator(
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True)
    datagen.fit(x_train)

skipping augmentation


**Create Model**

In [15]:
model = Sequential()
model.add(Conv2D(30, kernel_size=(10, 10), strides=(1, 1), activation='relu', input_shape=[100,100,3]))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(60, (5, 5), activation='relu', name='second_conv',strides=(1, 1)))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(90, (3, 3), activation='relu', name='third_conv',strides=(1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(2, activation='softmax'))
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=sgd, metrics=['accuracy'])

print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 91, 91, 30)        9030      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 45, 45, 30)        0         
_________________________________________________________________
second_conv (Conv2D)         (None, 41, 41, 60)        45060     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 60)        0         
_________________________________________________________________
third_conv (Conv2D)          (None, 11, 11, 90)        48690     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 90)          0         
_________________________________________________________________
flat

**Train**

Save checkpoints and jsons to file

In [0]:
currentDT = datetime.datetime.now()
model_path = '/content/drive/My Drive/vision_intro_tutorial/'+currentDT.strftime("%Y-%m-%d-%H:%M:%S")+'--checkpoint.hdf5'
model_json_path = '/content/drive/My Drive/vision_intro_tutorial/'+currentDT.strftime("%Y-%m-%d-%H:%M:%S")+'--model.json'
checkpoint = ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_acc', mode='max')
model_json = model.to_json()
with open(model_json_path, "w") as json_file:
    json_file.write(model_json)

Start training

In [0]:
if Augment is True:
  history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=20),
                    validation_data=(x_test, y_test), callbacks=[checkpoint],
                    steps_per_epoch=len(x_train) / 64, epochs=100)

In [18]:
if Augment is False:
  history = model.fit(x_train, y_train,
          batch_size=16,
          epochs=100,
          verbose=1,
          validation_data=(x_valid, y_valid),
          callbacks=[checkpoint])

Instructions for updating:
Use tf.cast instead.
Train on 279 samples, validate on 81 samples
Epoch 1/100
279/279 [==============================] - 6s 21ms/step - loss: 0.6845 - acc: 0.5556 - val_loss: 0.7504 - val_acc: 0.3086
Epoch 2/100
279/279 [==============================] - 0s 749us/step - loss: 0.6777 - acc: 0.5556 - val_loss: 0.7235 - val_acc: 0.3086
Epoch 3/100
279/279 [==============================] - 0s 731us/step - loss: 0.6738 - acc: 0.5556 - val_loss: 0.7312 - val_acc: 0.3086
Epoch 4/100
279/279 [==============================] - 0s 724us/step - loss: 0.6644 - acc: 0.5556 - val_loss: 0.6947 - val_acc: 0.3086
Epoch 5/100
279/279 [==============================] - 0s 761us/step - loss: 0.6682 - acc: 0.6057 - val_loss: 0.7140 - val_acc: 0.3086
Epoch 6/100
279/279 [==============================] - 0s 739us/step - loss: 0.6533 - acc: 0.6344 - val_loss: 0.7512 - val_acc: 0.3086
Epoch 7/100
279/279 [==============================] - 0s 745us/step - loss: 0.6480 - acc: 0.6452 

In [22]:
model.load_weights(filepath=model_path)
y_predicted = model.predict_classes(x_test, verbose = 0)
y_actu = pd.Series(y_test_r, name='Actual')
y_pred = pd.Series(y_predicted, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
print(df_confusion)
class_accuracy_percentage = class_accuracy_percentage_calc(y_actu,y_pred)
print('Accuracy %:')
print(class_accuracy_percentage)

Predicted   0   1  All
Actual                
0          18   2   20
1           2  18   20
All        20  20   40
Accuracy %:
[0.9, 0.9]
